### Import Statments 

In [41]:
import pandas as pd
import keras
import keras_tuner
from keras import layers
from keras_tuner import RandomSearch
from sklearn.model_selection import train_test_split

### Import Data

In [65]:
df = pd.read_csv('Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [69]:
# drop null values
df.dropna(inplace=True)

In [71]:
# split dataset into X and y 
X = df.iloc[:,:-1]
y = df.iloc[:, -1]

In [72]:
# split dataset into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [73]:
# build model
def build_model(hp):
    # instantiate sequential model
    model = keras.Sequential()
    
    # tune the number of layers
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(
            layers.Dense(
                # Tune number of units separately
                units=hp.Int(f"unit_{i}", min_value=32, max_value=512, step=32),
                activation='relu'),
            )
    # Add final output layer
    model.add(layers.Dense(units=1, activation='linear'))
    
    model.compile(optimizer=keras.optimizers.Adam( # Tune the learning rate to use
                                                    learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
                    ),
                  loss='mean_absolute_error',
                  metrics=[keras.metrics.MeanAbsoluteError()]
                 )
    return model

In [74]:
# Initialize tuner
tuner = RandomSearch(
        hypermodel=build_model,
        objective=keras_tuner.Objective("val_mean_absolute_error", direction="min"),
        max_trials=5,
        executions_per_trial=3,
        overwrite=True,
        directory='project1',
        project_name='Air Quality Index',
        )

In [75]:
# Print summary of search space
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
unit_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
unit_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [76]:
# start search for best parameter
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 14s]
val_mean_absolute_error: 48.58720143636068

Best val_mean_absolute_error So Far: 44.03432846069336
Total elapsed time: 00h 01m 01s
INFO:tensorflow:Oracle triggered exit


In [77]:
# print search results
tuner.results_summary()

Results summary
Results in project1\Air Quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 6
unit_0: 192
unit_1: 160
learning_rate: 0.01
unit_2: 64
unit_3: 512
unit_4: 480
unit_5: 64
unit_6: 384
unit_7: 128
unit_8: 224
unit_9: 416
unit_10: 384
unit_11: 192
unit_12: 256
unit_13: 256
unit_14: 288
unit_15: 160
unit_16: 128
unit_17: 224
unit_18: 416
Score: 44.03432846069336
Trial summary
Hyperparameters:
num_layers: 14
unit_0: 480
unit_1: 160
learning_rate: 0.01
unit_2: 448
unit_3: 224
unit_4: 320
unit_5: 192
unit_6: 160
unit_7: 384
unit_8: 64
unit_9: 416
unit_10: 352
unit_11: 192
unit_12: 480
unit_13: 256
unit_14: 64
unit_15: 96
unit_16: 448
unit_17: 256
unit_18: 256
Score: 48.58720143636068
Trial summary
Hyperparameters:
num_layers: 12
unit_0: 384
unit_1: 256
learning_rate: 0.001
unit_2: 384
unit_3: 96
unit_4: 32
unit_5: 32
unit_6: 32
unit_7: 32
unit_8: 32
unit_9: 32
unit_10: 32
unit_11: 32
Score: 59.95540110270182
Trial summary
Hyperparameters:
num_layers: 4

In [85]:
# get the best models
models = tuner.get_best_models()

# get the best model
best_model = models[0]

In [87]:
# build best model 
best_model.build(X_train.shape)

In [88]:
# print summary of model
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (764, 192)                1728      
                                                                 
 dense_1 (Dense)             (764, 160)                30880     
                                                                 
 dense_2 (Dense)             (764, 64)                 10304     
                                                                 
 dense_3 (Dense)             (764, 512)                33280     
                                                                 
 dense_4 (Dense)             (764, 480)                246240    
                                                                 
 dense_5 (Dense)             (764, 64)                 30784     
                                                                 
 dense_6 (Dense)             (764, 1)                  6